# Live Voltage Output (Moderate)
This example will create a model where neuron voltage is output from the network.

Create a device that extends ```spynnaker.pyNN.external_devices_models.AbstractMulticastControllableDevice```, and adds all the required properties therein (```device_control_partition_id```, ```device_control_key```, ```device_control_uses_payload```, ```device_control_min_value```, ```device_control_max_value```, ```device_control_timesteps_between_sending```, ```device_control_send_type``` and ```device_control_scaling_factor```).  This should have an acceptable voltage range of -50 to 50 and send the value as an integer every 5 timesteps of the simulation, and should use a key of 10.

In [1]:
from spynnaker.pyNN.external_devices_models import AbstractMulticastControllableDevice
from spynnaker.pyNN.external_devices_models.abstract_multicast_controllable_device import SendType

class TestDevice(AbstractMulticastControllableDevice):
    
    @property
    def device_control_partition_id(self):
        return "Test"
    
    @property
    def device_control_key(self):
        return 0
    
    @property
    def device_control_uses_payload(self):
        return True
    
    @property
    def device_control_min_value(self):
        return -50
    
    @property
    def device_control_max_value(self):
        return 50
    
    @property
    def device_control_timesteps_between_sending(self):
        return 5
    
    @property
    def device_control_send_type(self):
        # The type of the value - one of the SendType values
        return SendType.SEND_TYPE_UINT
    
    @property
    def device_control_scaling_factor(self):
        # The amount to multiply the voltage by before transmission
        return 1.0

Create a translator that extends ```spynnaker.pyNN.external_devices_models.AbstractEthernetTranslator```, and implement the ```translate_control_packet``` method.  This accepts a packet, which has the properties ```key``` and ```payload```.  Print the received key, and the voltage value from the payload.

In [2]:
from spynnaker.pyNN.external_devices_models import AbstractEthernetTranslator

class Translator(AbstractEthernetTranslator):
    
    def translate_control_packet(self, packet):
        print("Received Key {}: Voltage {}".format(packet.key, packet.payload))

Setup the simulation to run at 1ms timesteps.

In [3]:
import pyNN.spiNNaker as sim

sim.setup(1.0)

2020-01-16 17:37:17 INFO: Read cfg files: /home/jovyan/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/jovyan/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg, /home/jovyan/.spynnaker.cfg
2020-01-16 17:37:17 INFO: Will search these locations for binaries: /home/jovyan/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/common_model_binaries : /home/jovyan/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/model_binaries
2020-01-16 17:37:17 INFO: Setting time scale factor to 1.0.
2020-01-16 17:37:17 INFO: Setting machine time step to 1000 micro-seconds.


Detected PyNN version 0.9.4 and Neo version 0.8.0
['/home/jovyan/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/jovyan/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/jovyan/.spynnaker.cfg']


0

Set up a Population of 100 Poisson neurons spiking at a rate of 10Hz.

In [4]:
source = sim.Population(100, sim.SpikeSourcePoisson(rate=10), label="source")

Set up an External Control Population with 1 neuron using the device and translator created above.

In [5]:
pop = sim.external_devices.EthernetControlPopulation(
    n_neurons=1,
    model=sim.external_devices.ExternalDeviceLifControl(
        devices=[TestDevice()],
        create_edges=False,
        translator=Translator()))

Connect every neuron in the Poisson source to the control population with a weight of 0.1nA and a delay of 1ms. 

In [6]:
sim.Projection(source, pop, sim.AllToAllConnector(), sim.StaticSynapse(weight=0.1, delay=1))

projection from pre source to post Population 0 with connector AllToAllConnector()

Run the simulation for 1000ms, and then end the simulation.  Watch to see the messages produced.

In [7]:
sim.run(1000)
sim.end()

2020-01-16 17:37:17 INFO: Simulating for 1 1000ms timesteps using a hardware timestep of 1000000us
2020-01-16 17:37:17 INFO: Starting execution process
2020-01-16 17:37:20 INFO: Time 0:00:03.549554 taken by SpallocMaxMachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2020-01-16 17:37:30 INFO: Time 0:00:09.385723 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2020-01-16 17:37:36 INFO: Time 0:00:05.854011 taken by PartitionAndPlacePartitioner
Created spalloc job 5245528
2020-01-16 17:37:36 INFO: Created spalloc job 5245528
Waiting for board power commands to complete.
2020-01-16 17:37:36 INFO: Waiting for board power commands to complete.
2020-01-16 17:37:41 INFO: Time 0:00:05.048369 taken by SpallocAllocator
2020-01-16

2020-01-16 17:37:56 INFO: Time 0:00:00.150737 taken by HostExecuteSystemDataSpecification
Loading system executables onto the machine
|0%                          50%                         100%|
2020-01-16 17:38:02 INFO: Time 0:00:05.723420 taken by LoadSystemExecutableImages
Clearing tags
|0%                          50%                         100%|
Loading Tags
|0%                          50%                         100%|
2020-01-16 17:38:02 INFO: Time 0:00:00.022213 taken by TagsLoader
Writing data
|0%                          50%                         100%|
2020-01-16 17:38:02 INFO: Time 0:00:00.031242 taken by WriteMemoryIOData
Executing data specifications and loading data for application vertices
|0%                          50%                         100%|
2020-01-16 17:38:02 INFO: Time 0:00:00.144426 taken by HostExecuteApplicationDataSpecification
Expanding Synapses
|0%                          50%                         100%|
2020-01-16 17:38:06 INFO: Time 0:00:03.73

# Extension 1
Add a live Poisson rate control.  Try to stablise the membrane voltage at 25Hz by altering the rate of the Poisson source based on the live voltage output.

In [9]:
from spynnaker.pyNN.external_devices_models import AbstractMulticastControllableDevice
from spynnaker.pyNN.external_devices_models.abstract_multicast_controllable_device import SendType


class TestDevice(AbstractMulticastControllableDevice):

    @property
    def device_control_partition_id(self):
        return "Test"

    @property
    def device_control_key(self):
        return 0

    @property
    def device_control_uses_payload(self):
        return True

    @property
    def device_control_min_value(self):
        return -50

    @property
    def device_control_max_value(self):
        return 50

    @property
    def device_control_timesteps_between_sending(self):
        return 5

    @property
    def device_control_send_type(self):
        # The type of the value - one of the SendType values
        return SendType.SEND_TYPE_UINT

    @property
    def device_control_scaling_factor(self):
        # The amount to multiply the voltage by before transmission
        return 1.0

from spynnaker.pyNN.external_devices_models import AbstractEthernetTranslator

class Translator(AbstractEthernetTranslator):

    def __init__(self, connection):
        self._connection = connection
        self._rate = 10
        self._average_voltage = 0

    def translate_control_packet(self, packet):
        print("Received Key {}: Voltage {}".format(packet.key, packet.payload))
        self._average_voltage = ((self._average_voltage * 9) + packet.payload) / 10
        diff = (self._average_voltage - 25) / 50
        new_rate = self._rate - diff
        if new_rate < 0:
            new_rate = 0
        if new_rate != self._rate:
            print("Setting rate to {}".format(new_rate))
            self._rate = new_rate
            self._connection.set_rates("source", [(i, new_rate) for i in range(100)])

import pyNN.spiNNaker as sim

connection = sim.external_devices.SpynnakerPoissonControlConnection(local_port=None, poisson_labels=["source"])

sim.setup(1.0)

source = sim.Population(100, sim.SpikeSourcePoisson(rate=10), label="source")
pop = sim.external_devices.EthernetControlPopulation(
    n_neurons=1,
    model=sim.external_devices.ExternalDeviceLifControl(
        devices=[TestDevice()],
        create_edges=False,
        translator=Translator(connection)))

sim.Projection(source, pop, sim.AllToAllConnector(), sim.StaticSynapse(weight=0.1, delay=1))

sim.external_devices.add_poisson_live_rate_control(source, database_notify_port_num=connection.local_port)

sim.run(1000)
sim.end()

2020-01-17 10:25:58 INFO: 0.0.0.0:43862 Waiting for message to indicate that the database is ready
2020-01-17 10:25:58 INFO: Read cfg files: /home/jovyan/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/jovyan/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg, /home/jovyan/.spynnaker.cfg
2020-01-17 10:25:58 INFO: Will search these locations for binaries: /home/jovyan/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/common_model_binaries : /home/jovyan/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/model_binaries
2020-01-17 10:25:58 INFO: Setting time scale factor to 1.0.
2020-01-17 10:25:58 INFO: Setting machine time step to 1000 micro-seconds.
2020-01-17 10:25:58 INFO: Simulating for 1 1000ms timesteps using a hardware timestep of 1000000us
2020-01-17 10:25:58 INFO: Starting execution process


['/home/jovyan/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/jovyan/sPyNNaker/lib/python3.6/site-packages/spynnaker/pyNN/spynnaker.cfg', '/home/jovyan/.spynnaker.cfg']


2020-01-17 10:26:01 INFO: Time 0:00:03.434311 taken by SpallocMaxMachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2020-01-17 10:26:11 INFO: Time 0:00:09.761511 taken by PreAllocateResourcesForExtraMonitorSupport
Partitioning graph vertices
|0%                          50%                         100%|
Partitioning graph edges
|0%                          50%                         100%|
2020-01-17 10:26:17 INFO: Time 0:00:06.083165 taken by PartitionAndPlacePartitioner
Created spalloc job 5245817
2020-01-17 10:26:17 INFO: Created spalloc job 5245817
Waiting for board power commands to complete.
2020-01-17 10:26:17 INFO: Waiting for board power commands to complete.
2020-01-17 10:26:22 INFO: Time 0:00:05.051109 taken by SpallocAllocator
2020-01-17 10:26:22 INFO: Creating transceiver for 10.11.195.81
2020-01-17 10:26:22 INFO: Working out if machine is booted
2020-01-17 10:26:26 INFO: Attempting to

2020-01-17 10:26:37 INFO: Time 0:00:00.154417 taken by HostExecuteSystemDataSpecification
Loading system executables onto the machine
|0%                          50%                         100%|
2020-01-17 10:26:43 INFO: Time 0:00:05.721692 taken by LoadSystemExecutableImages
Clearing tags
|0%                          50%                         100%|
Loading Tags
|0%                          50%                         100%|
2020-01-17 10:26:43 INFO: Time 0:00:00.018884 taken by TagsLoader
Writing data
|0%                          50%                         100%|
2020-01-17 10:26:43 INFO: Time 0:00:00.026522 taken by WriteMemoryIOData
Executing data specifications and loading data for application vertices
|0%                          50%                         100%|
2020-01-17 10:26:43 INFO: Time 0:00:00.148864 taken by HostExecuteApplicationDataSpecification
Expanding Synapses
|0%                          50%                         100%|
2020-01-17 10:26:47 INFO: Time 0:00:03.72